# Guides

## How to customize or extend the knowledge base

You can design your own description language and use it with Draco or extend the existing language we use here. If you don't know where to start with the constraints, you can first use our [`run_clingo`](https://dig.cmu.edu/draco2/api/run.html#draco.run.run_clingo) and [`programs`](https://dig.cmu.edu/draco2/api/programs.html) API to generate some recommendations. Then, you should be able to find some recommendations that should have been left out, and you can write constraints to reflect them.  
If you write your own description language, you need to set up the search space in a similar way to [`generate.lp`](https://github.com/cmudig/draco2/blob/main/draco/asp/generate.lp) before trying to generate recommendations.

For example, the following snippet shows how to use the `run_clingo` API to generate one recommendation. You can set a different number to look into more results. 

In [4]:
from draco import answer_set_to_dict, run_clingo
from draco.programs import define, hard, helpers, constraints, generate
from pprint import pprint

prog = (
    generate.program
    + define.program
    + helpers.program
    + hard.program
    + constraints.program
)

scatter = """
    attribute(number_rows,root,100).

    entity(field,root,(f,0)).
    attribute((field,name),(f,0),temperature).
    attribute((field,type),(f,0),number).

    entity(field,root,(f,1)).
    attribute((field,name),(f,1),precipitation).
    attribute((field,type),(f,1),number).

    entity(view,root,(v,0)).
    entity(mark,(v,0),(m,0)).
    entity(encoding,(m,0),(e,0)).
    attribute((encoding,field),(e,0),(f,0)).
    entity(encoding,(m,0),(e,1)).
    attribute((encoding,field),(e,1),(f,1)).

    entity(scale,(v,0),(s,0)).
    entity(scale,(v,0),(s,1)).

    #show entity/3.
    #show attribute/3.
"""

for model in run_clingo(prog + scatter, 1):
    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)

{'field': [{'name': 'temperature', 'type': 'number'},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'value',
 'view': [{'mark': [{'channel': 'text',
                     'encoding': [{'channel': 'text',
                                   'field': ('f', 0),
                                   'scale_type': 'linear'},
                                  {'channel': 'size',
                                   'field': ('f', 1),
                                   'scale_type': 'linear'}],
                     'mark_encoding_cont': ('e', 1),
                     'scale': ('s', 1),
                     'type': 'text'}],
           'scale': [{'channel': 'text', 'type': 'linear'},
                     {'channel': 'size', 'type': 'linear'}]}]}
[attribute(number_rows,root,100), attribute((field,name),(f,0),temperature), attribute((field,type),(f,0),number), attribute((field,name),(f,1),precipitation), attribute((field,type),(f,1),number), attribute((encoding,

If you see that there are too many recommendations, you can:
 * add more hard constraints
 * modify your generator and hard constraints to reduce symmetry in the search space (e.g. similar recommendations with switched entity ids)


If you see too few recommendations, you can:
 * check if some of your constraints are too tight, and move them to soft constraints
 

If you see no recommendations, you might have made mistakes in the hard constraints. You can allow violations to check what are the common ones by removing the `violation` constraint, which forbids any violations, from the programs. Below is an example:

In [5]:
from draco import Draco
from draco.asp_utils import blocks_to_program

d = Draco()

c = "".join(
    blocks_to_program(
        constraints.blocks, set(constraints.blocks.keys()) - set(["violation"])
    )
)
prog = generate.program + define.program + helpers.program + hard.program + c + scatter

for model in run_clingo(prog + scatter, 1):
    pprint(answer_set_to_dict(model.answer_set))
    print(model.answer_set)
    answer = [str(symbol) + ". " for symbol in model.answer_set]
    print(d.get_violations(answer))

{'field': [{'name': 'temperature', 'type': 'number'},
           {'name': 'precipitation', 'type': 'number'}],
 'number_rows': 100,
 'task': 'value',
 'view': [{'mark': [{'channel': 'shape',
                     'encoding': [{'channel': 'shape',
                                   'field': ('f', 0),
                                   'scale_type': 'categorical'},
                                  {'channel': 'size',
                                   'field': ('f', 1),
                                   'scale_type': 'categorical'}],
                     'mark_channel_discrete_or_binned': 'shape',
                     'mark_encoding_discrete_or_binned': ('e', 1),
                     'scale': ('s', 1),
                     'type': 'rect'}],
           'scale': [{'channel': 'shape', 'type': 'categorical'},
                     {'channel': 'size', 'type': 'categorical'}]}]}
[attribute(number_rows,root,100), attribute((field,name),(f,0),temperature), attribute((field,type),(f,0),number), a